In [ ]:
!pip install -r https://raw.githubusercontent.com/GoogleCloudPlatform/python-docs-samples/main/people-and-planet-ai/land-cover-classification/requirements.txt

In [ ]:
#@title My Google Cloud resources
project = "" #@param {type:"string"}
bucket = "" #@param {type:"string"}
location = "us-central1" #@param {type:"string"}

# Load resources from environment variables if available
import os
project = os.environ.get('GOOGLE_CLOUD_PROJECT', project)
bucket = os.environ.get('CLOUD_STORAGE_BUCKET', bucket)
location = os.environ.get('CLOUD_LOCATION', location)

# Quick input validations
assert project, '⚠️ Please provide a Google Cloud project ID'
assert bucket, '⚠️ Please provide a Cloud Storage bucket name'
assert not bucket.startswith('gs://'), f'⚠️ Please remove the gs:// prefix from the bucket name: {bucket}'
assert location, '⚠️ Please provide a Google Cloud location'

# Authenticate
from google.colab import auth
auth.authenticate_user()

In [ ]:
import ee
import google.auth

credentials, _ = google.auth.default()
ee.Initialize(credentials, project=project)

In [ ]:
CLASSIFICATIONS = {
    '💧 Water':              '449cda',
    '🌳 Trees':              '3d7c49',
    '🌾 Grass':              '8ab052',
    '🌿 Flooded vegetation': '7c86c7',
    '🚜 Crops':              'e19832',
    '🪴 Shrub and scrub':    'dfc35b',
    '🏗️ Built-up areas':     'c52918',
    '🪨 Bare ground':        'a29d90',
    '❄️ Snow and ice':       'b4a0e0',
}
PALETTE = list(CLASSIFICATIONS.values())

def display_legend(classifications):
  reset_color = '\u001b[0m'
  colored = lambda red, green, blue: f"\033[48;2;{red};{green};{blue}m"
  for name, color in classifications.items():
    red   = int(color[0:2], 16)
    green = int(color[2:4], 16)
    blue  = int(color[4:6], 16)
    print(f"{colored(red, green, blue)}   {reset_color} {name}")

display_legend(CLASSIFICATIONS)

In [ ]:
import folium

import export_data

sentinel2_image = export_data.get_sentinel2_image('2020-1-1', '2021-01-1')

vis_params = {
  'min': 0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'],
}
folium.Map(
    location=[38.6561723, -9.0624997],
    zoom_start=12,
    tiles=sentinel2_image.getMapId(vis_params)['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
)

In [ ]:
import folium

import export_data

landcover_image = export_data.get_landcover_image()

vis_params = {
  'bands': ['landcover'],
  'palette': PALETTE,
}
folium.Map(
    location=[38.6561723, -9.0624997],
    zoom_start=12,
    tiles=landcover_image.getMapId(vis_params)['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
)

In [ ]:
#@title
points_per_class = 10  #@param {type:"integer"}
padding = 15  #@param {type:"integer"}

# Load from environment variables if available
points_per_class = int(os.environ.get('POINTS_PER_CLASS', points_per_class))
padding = int(os.environ.get('PADDING', padding))

In [ ]:
import export_data

export_data.run(
    bucket=bucket,
    training_prefix="land-cover/training",
    validation_prefix="land-cover/validation",
    points_per_class=points_per_class,
    padding=padding,
)

In [ ]:
import tensorflow as tf

filename = f"gs://{bucket}/land-cover/training.tfrecord.gz"
dataset = tf.data.TFRecordDataset(filename, compression_type='GZIP') 
for x in dataset.take(1):
  example = tf.train.Example()
  example.ParseFromString(x.numpy())
  for _, feature in example.ListFields():
    for _, field in feature.ListFields():
      for name, values in sorted(field.items()):
        print(f"{name}: float[{len(values.float_list.value)}]")